In [1]:
import xml.etree.ElementTree as ET

import pandas as pd
from datetime import datetime



In [2]:
ugly = '/home/tony/opt/dvr1/5_tv_guide/xmltv.xml'


In [3]:
def tv_df_make(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    # create channel lookup table
    chans_list = []
    for chan_elem in root.findall('.//channel'):
        chan_dict = {}
        chan_dict['id'] = chan_elem.get('id')
        i=0
        for child_elem in chan_elem.findall('display-name'):
            #print (child_elem.text)
            tag=child_elem.tag+str(i)
            chan_dict[tag] = child_elem.text
            i += 1
        chans_list.append(chan_dict)
    CHAN_DF = pd.DataFrame(chans_list)
    CHAN_DICT = CHAN_DF.to_dict('records')
    CHAN_LOOKUP={}
    for rec in CHAN_DICT:
        CHAN_LOOKUP[rec['id']] = rec['display-name1']
    
    
    # Create a list to store dictionaries representing each book
    progr_list = []
    # Iterate through each <book> element
    for progr_elem in root.findall('.//programme'):
        progr_dict = {}
        progr_dict['start'] = progr_elem.get('start')
        progr_dict['stop'] = progr_elem.get('stop')
        progr_dict['channel'] = progr_elem.get('channel')
        #progr_dict['title'] = progr_elem.get('title')
    
    
        i=0
        for child_elem in progr_elem.findall('title'):
            # print (child_elem.text)
            tag=child_elem.tag
            progr_dict[tag] = child_elem.text
            i += 1
            
        for child_elem in progr_elem.findall('sub-title'):
            # print (child_elem.text)
            tag=child_elem.tag
            progr_dict[tag] = child_elem.text
            
        for child_elem in progr_elem.findall('length'):
            # print (child_elem.text)
            tag=child_elem.tag
            progr_dict[tag] = child_elem.text
    
    
        progr_list.append(progr_dict)
    pdf = pd.DataFrame(progr_list)
    pdf['vchan'] = pdf['channel'].map(CHAN_LOOKUP)
    pdf['start'] = pd.to_datetime(pdf['start'])
    pdf['stop'] = pd.to_datetime(pdf['stop'])
    pdf['vchan'] = pdf['vchan'].astype(float) 


    now = datetime.now()
    local_timezone = now.astimezone().tzinfo

    print("Current timezone:", local_timezone)
    
    pdf['start'] = pdf['start'].dt.tz_convert(local_timezone)
    pdf['stop'] = pdf['stop'].dt.tz_convert(local_timezone)
    #pdf['start'] = pdf['start'].dt.tz_localize(local_timezone) 
    #pdf['stop'] = pdf['stop'].dt.tz_localize(local_timezone) 



    # Display the DataFrame
    print(pdf.head())
    return pdf
    

In [4]:
xml_file = ugly
df = tv_df_make(xml_file)

Current timezone: CST
                      start                      stop                channel  \
0 2025-01-18 15:00:00-06:00 2025-01-18 15:30:00-06:00  I2.1.44246.zap2it.com   
1 2025-01-18 15:30:00-06:00 2025-01-18 16:00:00-06:00  I2.1.44246.zap2it.com   
2 2025-01-18 16:00:00-06:00 2025-01-18 16:30:00-06:00  I2.1.44246.zap2it.com   
3 2025-01-18 16:30:00-06:00 2025-01-18 17:00:00-06:00  I2.1.44246.zap2it.com   
4 2025-01-18 17:00:00-06:00 2025-01-18 17:30:00-06:00  I2.1.44246.zap2it.com   

                             title                               sub-title  \
0                      Epic Trails   Backpacking the Grand Canyon - Part 2   
1  Samantha Brown's Places to Love                  New Orleans, Louisiana   
2              America's Heartland                                     NaN   
3         New Scandinavian Cooking                         A Coastal Feast   
4              Rick Steves' Europe  Rick Steves' Europe: Ancient Roman Art   

  length  vchan  
0     30  

In [5]:
df.columns

Index(['start', 'stop', 'channel', 'title', 'sub-title', 'length', 'vchan'], dtype='object')

In [6]:
type(df)

pandas.core.frame.DataFrame

In [7]:
df.dtypes

start        datetime64[ns, CST]
stop         datetime64[ns, CST]
channel                   object
title                     object
sub-title                 object
length                    object
vchan                    float64
dtype: object

In [8]:
# Get current time

now = datetime.now()
local_timezone = now.astimezone().tzinfo


now_aware = datetime.now(local_timezone)
now = now_aware
#now = datetime.now()

# Filter DataFrame to find rows with 'now' between 'start' and 'stop'
filtered_df = df[(df['start'] <= now) & (now <= df['stop'])]

filtered_df.head()


,start,stop,channel,title,sub-title,length,vchan
2,2025-01-18 16:00:00-06:00,2025-01-18 16:30:00-06:00,I2.1.44246.zap2it.com,America's Heartland,NaN,30,2.1
7,2025-01-18 16:00:00-06:00,2025-01-18 16:30:00-06:00,I2.2.44248.zap2it.com,To the Contrary With Bonnie Erbé,NaN,30,2.2
13,2025-01-18 16:00:00-06:00,2025-01-18 17:00:00-06:00,I2.3.62638.zap2it.com,Richard Bangs' Adventures With Purpose -- Swit...,NaN,60,2.3
18,2025-01-18 16:00:00-06:00,2025-01-18 16:30:00-06:00,I2.4.101941.zap2it.com,Odd Squad,Disorder in the Court,30,2.4
22,2025-01-18 14:00:00-06:00,2025-01-18 18:00:00-06:00,I2.6.108220.zap2it.com,Classical 24,NaN,240,2.6


In [9]:
df.columns.values

array(['start', 'stop', 'channel', 'title', 'sub-title', 'length',
       'vchan'], dtype=object)

In [10]:
df = filtered_df

In [11]:
# Sort the DataFrame by 'vchan'
df_sorted = df.sort_values(by='vchan')

# Select only the 'vchan', 'start', and 'title' columns
df_pruned = df_sorted[['vchan', 'start', 'title']]

# Print the resulting DataFrame
print(df_pruned) 

     vchan                     start  \
2      2.1 2025-01-18 16:00:00-06:00   
7      2.2 2025-01-18 16:00:00-06:00   
13     2.3 2025-01-18 16:00:00-06:00   
18     2.4 2025-01-18 16:00:00-06:00   
22     2.6 2025-01-18 14:00:00-06:00   
..     ...                       ...   
238   46.3 2025-01-18 16:00:00-06:00   
240   46.4 2025-01-18 16:00:00-06:00   
243   46.5 2025-01-18 16:00:00-06:00   
246   46.6 2025-01-18 16:00:00-06:00   
250   53.0 2025-01-18 16:00:00-06:00   

                                                 title  
2                                  America's Heartland  
7                     To the Contrary With Bonnie Erbé  
13   Richard Bangs' Adventures With Purpose -- Swit...  
18                                           Odd Squad  
22                                        Classical 24  
..                                                 ...  
238                                              Roots  
240                                               Monk  
243   

In [12]:
df_pruned.head(30)

,vchan,start,title
2,2.10,2025-01-18 16:00:00-06:00,America's Heartland
7,2.20,2025-01-18 16:00:00-06:00,To the Contrary With Bonnie Erbé
13,2.30,2025-01-18 16:00:00-06:00,Richard Bangs' Adventures With Purpose -- Swit...
18,2.40,2025-01-18 16:00:00-06:00,Odd Squad
22,2.60,2025-01-18 14:00:00-06:00,Classical 24
24,4.10,2025-01-18 16:00:00-06:00,Chasing Gold: Milan Cortina 2026
29,4.20,2025-01-18 16:00:00-06:00,Bloodline Detectives
33,4.30,2025-01-18 16:00:00-06:00,Have Gun -- Will Travel
37,4.40,2025-01-18 16:00:00-06:00,48 Hours
40,4.50,2025-01-18 16:00:00-06:00,Law & Order: Special Victims Unit


In [13]:
df_no_duplicates = df_pruned.drop_duplicates() 

In [14]:
df_no_duplicates.head(40)

,vchan,start,title
2,2.10,2025-01-18 16:00:00-06:00,America's Heartland
7,2.20,2025-01-18 16:00:00-06:00,To the Contrary With Bonnie Erbé
13,2.30,2025-01-18 16:00:00-06:00,Richard Bangs' Adventures With Purpose -- Swit...
18,2.40,2025-01-18 16:00:00-06:00,Odd Squad
22,2.60,2025-01-18 14:00:00-06:00,Classical 24
24,4.10,2025-01-18 16:00:00-06:00,Chasing Gold: Milan Cortina 2026
29,4.20,2025-01-18 16:00:00-06:00,Bloodline Detectives
33,4.30,2025-01-18 16:00:00-06:00,Have Gun -- Will Travel
37,4.40,2025-01-18 16:00:00-06:00,48 Hours
40,4.50,2025-01-18 16:00:00-06:00,Law & Order: Special Victims Unit


In [15]:
#ndf = df['vchan'].isin(['11.1', '11.2'])

In [16]:
sdf = df_no_duplicates
sdf['vchan'] = sdf['vchan'].astype(str)
#filtered_df = sdf[sdf['vchan'].contains(['11.', '13.'])]
filtered_df = sdf[sdf['vchan'].str.startswith('11.')]

/tmp/ipykernel_547654/2066567371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['vchan'] = sdf['vchan'].astype(str)


In [17]:
filtered_df

,vchan,start,title
93,11.1,2025-01-18 16:00:00-06:00,Hope in the Wild
99,11.2,2025-01-18 16:00:00-06:00,The Neighborhood
104,11.3,2025-01-18 16:00:00-06:00,Law & Order: Special Victims Unit
108,11.4,2025-01-18 16:00:00-06:00,Bloodline Detectives


In [18]:
filtered_df.dtypes

vchan                 object
start    datetime64[ns, CST]
title                 object
dtype: object

In [19]:
sdf

,vchan,start,title
2,2.1,2025-01-18 16:00:00-06:00,America's Heartland
7,2.2,2025-01-18 16:00:00-06:00,To the Contrary With Bonnie Erbé
13,2.3,2025-01-18 16:00:00-06:00,Richard Bangs' Adventures With Purpose -- Swit...
18,2.4,2025-01-18 16:00:00-06:00,Odd Squad
22,2.6,2025-01-18 14:00:00-06:00,Classical 24
...,...,...,...
238,46.3,2025-01-18 16:00:00-06:00,Roots
240,46.4,2025-01-18 16:00:00-06:00,Monk
243,46.5,2025-01-18 16:00:00-06:00,48 Hours
246,46.6,2025-01-18 16:00:00-06:00,Bones


In [20]:
import pandas as pd

# Sample DataFrame
data = {'vchan': ['ABC1', 'HBO2', 'CNN1', 'FOX3', '11.1', '11.2', '13.5', 'ABC4']}
df = pd.DataFrame(data)

# Convert 'vchan' to string (if not already)
df['vchan'] = df['vchan'].astype(str)

# Filter DataFrame using str.startswith()
filtered_df = df[df['vchan'].str.startswith('11.') | df['vchan'].str.startswith('13.')]

print(filtered_df)

  vchan
4  11.1
5  11.2
6  13.5


In [21]:
filtered_df.dtypes

vchan    object
dtype: object

In [22]:
fav_chans = [11,13,46]

In [23]:
dfs=[]
df = sdf
for chan in fav_chans:
    vmatch = f'{chan}.'
    print(vmatch)
    fdf = df[df['vchan'].str.startswith(vmatch)]
    #print(fdf)
    dfs.append(fdf)

11.
13.
46.


In [24]:
bdf = pd.concat(dfs)

In [25]:
bdf.head(20)

,vchan,start,title
93,11.1,2025-01-18 16:00:00-06:00,Hope in the Wild
99,11.2,2025-01-18 16:00:00-06:00,The Neighborhood
104,11.3,2025-01-18 16:00:00-06:00,Law & Order: Special Victims Unit
108,11.4,2025-01-18 16:00:00-06:00,Bloodline Detectives
113,13.1,2025-01-18 15:30:00-06:00,NFL Football
114,13.2,2025-01-18 14:30:00-06:00,Cheyenne Autumn
116,13.3,2025-01-18 16:00:00-06:00,Have Gun -- Will Travel
120,13.4,2025-01-18 16:00:00-06:00,"Meet, Marry, Murder"
123,13.5,2025-01-18 16:00:00-06:00,Rizzoli & Isles
127,13.6,2025-01-18 16:00:00-06:00,The Woody Woodpecker Show


In [26]:
fav_chans = [4,7,23,36, 44]

In [29]:
dfs=[]
df = sdf
for chan in fav_chans:
    vmatch = f'{chan}.'
    print(vmatch)
    fdf = df[df['vchan'].str.contains(vmatch)]
    #print(fdf)
    dfs.append(fdf)

4.
7.
23.
36.
44.


In [30]:
bdf = pd.concat(dfs)
bdf.head(38)

,vchan,start,title
24,4.1,2025-01-18 16:00:00-06:00,Chasing Gold: Milan Cortina 2026
29,4.2,2025-01-18 16:00:00-06:00,Bloodline Detectives
33,4.3,2025-01-18 16:00:00-06:00,Have Gun -- Will Travel
37,4.4,2025-01-18 16:00:00-06:00,48 Hours
40,4.5,2025-01-18 16:00:00-06:00,Law & Order: Special Victims Unit
132,14.1,2025-01-18 16:00:00-06:00,The Parkers
137,14.2,2025-01-18 16:00:00-06:00,CSI: Miami
139,14.3,2025-01-18 15:00:00-06:00,Mega Shark vs. Mecha Shark
142,14.4,2025-01-18 16:00:00-06:00,The Menendez Murders: Erik Tells All
211,44.0,2025-01-18 14:00:00-06:00,To Be Announced


In [31]:
fdf = df[df['title'].str.contains('48')]


In [32]:
fdf

,vchan,start,title
37,4.4,2025-01-18 16:00:00-06:00,48 Hours
243,46.5,2025-01-18 16:00:00-06:00,48 Hours
